In [1]:
from IPython.display import Image

# SQLite

SQLite는 서버의 필요 없이 DB의 파일에 기초하여 DB 처리를 구현한 임베디드 SQL DB 엔진입니다. 

SQLite는 별도의 설치 없이, 쉽고 편리하게 사용할 수 있다는 점에서 많이 사용되고 있습니다.

Mac OS X나 Linux에는 SQLite가 기본적으로 설치되어 있습니다. 파이썬은 버전 2.5 이상일 경우 SQLite 사용을 위한 모듈이 기본 내장되어 있습니다.



#### 파이썬과 DB

이썬과 DB를 서로 다른 대륙에 살고 있는 사람이라고 생각해 보겠습니다. 파이썬 대륙과 DB 대륙에 있는 사람이 함께 이야기를 하려면 어떻게 해야 할까요? 

전화, 이메일과 같은 방법들을 이용해 시도해볼 수 있겠죠. 이처럼 파이썬과 DB는 sqlite3라는 전화 중계기를 이용해 대화를 한답니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_ATL5wsL.max-800x600.png)



#### Python DB API


자, 그럼 파이썬과 DB가 서로 대화할 수 있도록 준비해 볼까요?

일단 파이썬에서 DB 연결을 위한 전화선인 sqlite3 모듈을 import해 보겠습니다.

In [2]:
import sqlite3
print("뿅💛")

뿅💛


sqlite3 모듈은 파이썬 표준 라이브러리로 SQLite에 대한 인터페이스를 기본적으로 제공합니다.

잠깐, 표준 라이브러리가 뭐냐구요? 

우리가 파이썬을 처음 설치할 때 기본적으로 설치되는 모듈이라고 이해하면 됩니다. 정리하면, 표준 라이브러리인 sqlite3로 DB를 쉽게 이용할 수 있습니다.

이제 파이썬과 DB를 연결해 보겠습니다. conn에 DB 이름을 정하여 입력합니다.

저는 mydb로 지었는데요. [이름].[확장자명]의 형태로 저장하면 됩니다.

In [3]:
import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
print(conn)

conn 객체에는 SQL 연결과 관련된 셋팅이 포함되어 있습니다. 

- 이번에는 Connect() 함수의 연결을 사용하는 새로운 Cursor 객체를 만듭니다.

In [4]:
c = conn.cursor()
print(c)

Cursor는 SQL 질의(Query)를 수행하고 결과를 얻는데 사용하는 객체입니다. 

INSERT처럼 DB에만 적용되는 명령어를 사용한다면 Cursor를 안 사용할 수 있지만 SELECT와 같이 데이터를 불러올 때는 SQL 질의 수행 결과에 접근하기 위한 Cursor가 반드시 필요합니다. 

이러한 이유로 습관적으로 conn.cursor()를 사용하는 것을 권해 드립니다.

참고로, 파이썬 공식 개발 가이드에서도 Cursor를 사용하는 것을 표준으로 안내하고 있습니다.

    - SQL INSERT문 : 테이블에 데이터를 삽입(추가)하는 질의입니다.
    
    - SQL SELECT문 : 테이블에 데이터를 조건에 따라 조회하는 질의입니다.

In [5]:
# stocks이라는 이름의 테이블을 하나 생성합니다. 혹시 이미 생성되었다면 생략합니다. 
c.execute("CREATE TABLE  IF NOT EXISTS  stocks (date text, trans text, symbol text, qty real, price real)")

# stocks 테이블에 데이터를 하나 인서트합니다. 
c.execute("INSERT INTO stocks VALUES ('20200701', 'TEST', 'AIFFEL', 1, 10000)")

# 방금 인서트한 데이터를 조회해 봅니다.
c.execute("SELECT * FROM stocks")

# 조회된 내역을 커서를 통해 가져와 출력해 봅니다.
print(c.fetchone())

('20200701', 'TEST', 'AIFFEL', 1.0, 10000.0)


방금 우리는 sqlite3 모듈을 이용해 데이터베이스에 테이블을 하나 만들고 데이터를 인서트 한 후, 그 데이터를 조회해 보는 아주 기본적인 DB 핸들링 시나리오를 수행해 보았습니다.

## commit

삽입, 갱신, 삭제 등의 SQL 질의가 끝났다면 conn.commit()를 호출해야 DB가 실제로 업데이트 됩니다 . 

commit()을 하기 전에는 DB에 데이터가 업데이트된 것 같아 보여도 임시로만 바뀐 것이니 주의해야 합니다. 

하지만 우리는 sqlite3를 이용해 데이터가 잘 인서트 되었음을 SELECT문의 결과를 통해서 이미 확인했습니다.

이것이 데이터베이스를 사용할 때의 주의사항입니다.

우리는 sqlite3를 이용해 데이터베이스에 connection을 하나 맺었습니다.

이후 해당 connection을 통해 인서트 된 데이터는 conn.commit()를 호출하기 전까지는 그 connection 안에서만 유효합니다.

그럼 원본 데이터에 실제로 적용하려면 어떻게 해야 할까요? 

이미 살짝 예상하셨을 것 같은데요. 아래와 같이 commit() 명령어를 이용하시면 된답니다. 

참고로 select처럼 데이터를 가져오기만 하는 질의문의 경우에는 commit()가 필요 없습니다. 데이터에 아무런 변경사항이 없었으니까요.

In [6]:
conn.commit()   # commit()은 cursor의 메소드가 아니라 connection의 메소드입니다. 
print("뿅💛")

뿅💛


commit()을 통해 데이터베이스에 데이터 변경이 실제적으로 반영되었습니다.

이런 것을 데이터베이스에서는 트랜잭션(transaction) 관리라고 합니다.

트랜잭션의 개념에 대해 아래 글을 읽고 간단히 정리해 봅시다.

   [트렌잭션이란](https://sjh836.tistory.com/11)
   
commit()을 완료했다면 DB와 대화하는 것을 마무리 지어야 합니다. 

대화의 마무리는 DB와의 연결을 끊는 것으로 이루어집니다. 바로 close()를 이용하면 되는데요. 한번 실행해 봅시다.

In [7]:
c.close()      # 먼저 커서를 닫은 후
conn.close()    # DB 연결을 닫아 줍니다.
print("뿅💛")

뿅💛


## DDL문으로 테이블 생성하기

- 다양한 테이블을 조회해 보기 앞서, 실제 예제 테이블들을 한번 생성해 보도록 하겠습니다. 테이블명과 컬럼명을 한글로도 지정할 수 있습니다.

In [8]:
import sqlite3
import os
db_path = os.getenv('HOME')+'/mydb.db'

conn = sqlite3.connect(db_path)
c = conn.cursor()

#- ! 재실행 시 테이블이 존재할 수 있으므로 아래처럼 해당 테이블들을 모두 지워줍니다.
c.execute("DROP TABLE IF EXISTS 도서대출내역")
c.execute("DROP TABLE IF EXISTS 도서대출내역2")
c.execute("DROP TABLE IF EXISTS 대출내역")
c.execute("DROP TABLE IF EXISTS 도서명")


#----- 1st table : 도서대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역 (ID varchar, 이름 varchar, 도서ID varchar, 대출일 varchar, 반납일 varchar)")
#- 생성(create)문 : 테이블명, 변수명, 변수타입을 지정

data = [('101','문강태','aaa','2020-06-01','2020-06-05'),
             ('101','문강태','ccc','2020-06-20','2020-06-25'),
             ('102','고문영','bbb','2020-06-01',None),
             ('102','고문영','ddd','2020-06-08',None),
             ('103','문상태','ccc','2020-06-01','2020-06-05'),
             ('104','강기둥',None,None,None)]
#- 입력할 데이터를 그대로 입력 (변수명 순서 기준대로)

c.executemany('INSERT INTO 도서대출내역 VALUES (?,?,?,?,?)', data)
#- 입력할 데이터를 실제 테이블에 insert하기
#-----------------------------------------------#


#----- 2nd table : 도서대출내역2 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역2 (ID varchar, 이름 varchar, 대출년월 varchar, 대출일수 varchar)")

data = [('101','문강태','2020-06','20일'),
             ('102','고문영','2020-06','10일'),
             ('103','문상태','2020-06','8일'),
             ('104','강기둥','2020-06','3일')]
c.executemany('INSERT INTO 도서대출내역2 VALUES (?,?,?,?)', data)
#--------------------------------------------------#


#----- 3rd table : 대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 대출내역 (ID varchar, 이름 varchar, 도서ID varchar)")

data = [('101','문강태','aaa'),
             ('102','고문영','bbb'),
             ('102','고문영','fff'),
             ('103','문상태','ccc'),
             ('104','강기둥',None)]
c.executemany('INSERT INTO 대출내역 VALUES (?,?,?)', data)
#-----------------------------------------#


#----- 4th table : 도서명 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

data = [('aaa','악몽을 먹고 자란 소년'),
             ('bbb','좀비아이'),
             ('ccc','공룡백과사전'),
             ('ddd','빨간구두'),
             ('eee','잠자는 숲속의 미녀')]

c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)
#--------------------------------------#

conn.commit()
conn.close()
print("뿅💛")

뿅💛


    # ----- 4th table : 도서명 -----#

    # [1]
    c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

    # [2]
    data = [('aaa','악몽을 먹고 자란 소년'),
                 ('bbb','좀비아이'),
                 ('ccc','공룡백과사전'),
                 ('ddd','빨간구두'),
                 ('eee','잠자는 숲속의 미녀')]

    # [3]
    c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)

    # --------------------------------------#



1번의 CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar) 코드를 주목해 봅시다. 

이 부분은 "만약 테이블이 존재하지 않으면 테이블을 생성해 주세요. 

그리고 변수명과 각 변수의 타입은 다음과 같이 설정해 주세요!"라는 의미입니다. 

예제에서 테이블은 도서명 테이블이겠죠?

2번의 코드는 각 변수명에 맞게 데이터를 실제로 생성하는 것이구요.

마지막 3번에서 INSERT INTO 도서명 VALES (?, ?) 부분은 각 테이블의 변수(도서ID, 도서명)에 2번에서 입력한 데이터(data)를 넣겠다는 뜻입니다.

이렇게 ?를 사용해서 데이터를 쿼리에 바인딩하는 기법을 매우 흔히 사용하니 잘 알아둡시다.

또 하나 눈에 띄는 부분은, c.executemany() 메소드입니다. 

이것은 한꺼번에 여러 개의 데이터 처리를 가능하게 해 줍니다. 

입력해야 할 데이터를 위와 같이 list로 관리하고 있는 경우 매우 유용합니다.



In [9]:
conn = sqlite3.connect(db_path)
c = conn.cursor()

for row in c.execute('SELECT * FROM 도서명'):
  print(row)

('aaa', '악몽을 먹고 자란 소년')
('bbb', '좀비아이')
('ccc', '공룡백과사전')
('ddd', '빨간구두')
('eee', '잠자는 숲속의 미녀')


## SQL의 기본

SQL이란 무엇을 의미하는 것일까요?

SQL의 약자는 Structured Query Language입니다. 구조화된... 쿼리... 언어?

조금 더 풀어서 써보자면 데이터베이스(DB)에서 데이터를 조회하고자 할 때 필요한 컴퓨터 언어라고 할 수 있겠네요.

DB라는 공간에 ‘정형화된’(일정한 형식으로 수집되는) 데이터가 차곡차곡 저장되어 있습니다. 이러한 DB를 특정 언어로 조회해서 가져오는데, 그때 사용하는 언어가 바로 SQL입니다.

테이블을 삽입하거나 삭제, 갱신, 조회하는 등 본인이 원하는 형태로 데이터를 만들어 가져올 수 있습니다. 

이번에는 주로 데이터를 조회하고 조작하는 부분에 대해 다뤄보도록 하겠습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_1_h3s3VGU.png)

위와 같이 DB에서 특정 테이블(도서대출내역)을 조회하고 그것을 가지고 올 때 사용한 SELECT * FROM 도서대출내역 이 바로 SQL입니다.

#### 쿼리의 기본 구조

- SELECT ~ : 조회할 컬럼명을 선택


- FROM ~ : 조회할 테이블명을 지정 (위치와 테이블명을 입력)


- WHERE ~ : 질의할 때 필요한 조건을 설정


- GROUP BY ~ : 특정 컬럼을 기준으로 그룹을 지어 출력


- ORDER BY ~ : SELECT 다음에 오는 컬럼 중 정렬이 필요한 부분을 정렬 (기본 설정 : 오름차순)


- LIMIT 숫자 : Display하고자 하는 행의 수를 설정

![title](https://d3s0tskafalll9.cloudfront.net/media/images/F-29-3A.max-800x600.png)
![title](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_3_esNgMRX.max-800x600.png)

### < 테이블 명 : 도서대출내역 >
![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_4_zyTZEE7.png)

'싸이코지만 괜찮아' 이 드라마의 등장인물을 담은 테이블을 만들어 보았습니다.

테이블을 보시면 도서관에서 관리하고 있는 DB라는 점을 알 수 있죠. 그리고 사람들이 각자 어떤 도서를 대출했는지, 언제 대출/반납했는지를 파악할 수가 있네요.

- ID : 대출을 한 사람의 ID


- 이름 : 대출자의 이름


- 도서ID : 대출한 도서의 ID


- 대출일 : 도서를 대출한 날짜


- 반납일 : 도서를 반납한 날짜

자세히 보시면, 이렇게 5개의 열:컬럼(column)이 있습니다. 반대로 가로 줄은 행:로우(row)라고 부르죠.

![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_5_XoeqyR5.png)

지금까지 설명한 기본 구조를 위의 테이블을 활용해 검증해 봅시다.



In [10]:
#- 참고 : 실제 조회를 해보시려면 아래처럼 쓰시면 됩니다.
#- c.execute() 괄호 안에 SQL문을 넣으시면 됩니다.

import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
c = conn.cursor()

for row in c.execute('SELECT * FROM 도서대출내역'):
  print(row)

#- ! 출력 시 'NULL' 대신 'None'으로 출력될 수 있으나 동일하게 이해하시면 됩니다.

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('104', '강기둥', None, None, None)


In [11]:
# 전체조회

for row in c.execute('SELECT * FROM 도서대출내역'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('104', '강기둥', None, None, None)


![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_7_FKSgb5e.png)

SELECT와 FROM 사이에는 특정 컬럼을 넣어 출력하곤 하는데요. 

위의 쿼리처럼 별(*)을 입력하게 되면 '테이블 전체를 다 가져와라'라는 명령어가 됩니다.

그럼 특정 컬럼을 지정해 볼까요?



In [12]:
for row in c.execute('SELECT ID FROM 도서대출내역'):
  print(row)

('101',)
('101',)
('102',)
('102',)
('103',)
('104',)


### |특정컬럼을 지정|
![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_8_U5zAppm.png)

SELECT와 FROM 사이에 ‘ID’를 넣으면 '전체 테이블 중에 ID 컬럼만 가져와라'라는 명령어가 됩니다.

### | 조건을 입력하기 |

In [13]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 이름 = "문강태";'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')


![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_9_YQ4562W.png)

일단 SELECT와 FROM 사이에 별(*)이 있으니 전체 컬럼을 모두 가져온다는 것이겠죠?

근데 위와 달라진 점이 바로 WHERE절이 추가되었다는 점입니다. 

WHERE절 이후에는 특정 조건을 입력할 수 있습니다. 

위에서는 이름 = "문강태"라고 적어 놓았는데요. ‘이름이 문강태인 행(row)만 가져와라’라는 의미입니다. 

그래서 위처럼 5개 컬럼(column) * 2개의 로우(row)가 출력되는 것이랍니다.

### | GROUP BY로 중복을 제거해 보기 |

SELECT 이름 FROM 도서대출내역

GROUP BY 이름;

In [14]:
for row in c.execute('SELECT 이름 FROM 도서대출내역 GROUP BY 이름;'):
  print(row)

('강기둥',)
('고문영',)
('문강태',)
('문상태',)


![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_10_YLwMSrF.png)

GROUP BY는 이름 그대로 데이터를 그룹화시키는 역할을 합니다. 

'가나다라' 순으로 재정렬해서 뱉어주는 것도 확인할 수 있네요.

GROUP BY는 보통 집계성 함수와 함께 사용되는데요. 이는 뒤에서 집계성 함수에 대해 다룰 때 좀 더 자세히 살펴보도록 하고 넘어가겠습니다.

### | DISTINCT로 중복을 제거해보기 |

SELECT DISTINCT 이름 FROM 도서대출내역;

In [15]:
for row in c.execute('SELECT DISTINCT 이름 FROM 도서대출내역;'):
  print(row)

('문강태',)
('고문영',)
('문상태',)
('강기둥',)


위 GROUP BY 결과와 비슷하죠?

DISTINCT를 특정 컬럼 앞에 쓰면 해당 컬럼 값 중에서 중복되는 값들을 모두 제거한 후 출력해 줍니다.

### | ORDER BY로 정렬해보기 |

SELECT * FROM 도서대출내역

ORDER BY ID ;

In [16]:
for row in c.execute('SELECT * FROM 도서대출내역 ORDER BY ID ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('104', '강기둥', None, None, None)


![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_12_WHxK4Ap.png)

ORDER BY 뒤에 특정 컬럼명을 적으면, 그 컬럼을 기준 값으로 정렬을 해서 보여줍니다. 

위 쿼리문에서 보시면 SELECT와 FROM 사이에 별(*)이 들어 있으므로 '전체 컬럼을 가져오되 ID를 기준으로 정렬해달라'라는 뜻이 됩니다.

일반적으로 ORDER BY 뒤에는 (ASC)가 생략돼 있습니다. 

오름차순이 기본적으로 설정되어 있다는 것이지요. 반대는 내림차순으로 DESC라고 씁니다.

## ----------------------------------------------

SELECT * FROM 도서대출내역 

ORDER BY ID DESC ;

In [17]:
for row in c.execute('SELECT * FROM 도서대출내역 ORDER BY ID DESC ;'):
  print(row)

('104', '강기둥', None, None, None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')


![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_13_kngfh2m.png)

위와의 차이가 보이시죠? 이렇게 적으면 ID를 기준으로 내림차순 정렬이 됩니다.

### | 몇개의 row만 조회하기 |

SELECT * FROM 도서대출내역 LIMIT 5 ;

In [18]:
for row in c.execute('SELECT * FROM 도서대출내역 LIMIT 5 ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_14_5wtwgMS.png)

LIMIT 구문 뒤에 숫자를 적으면 그 숫자만큼의 행(row)만 출력됩니다. 

LIMIT 구문은 주로 처음 테이블들을 조회할 때 그 테이블의 구조를 파악하고, 어떤 값들이 존재하는지를 샘플로 파악하고자 할 때 사용됩니다.

그럼 이제 아래의 코드를 직접 실행창에 실행시켜 볼까요?


## ----------------------------------------------

SELECT 이름, 대출일, 반납일   
FROM 도서대출내역   
ORDER BY 대출일 DESC   
LIMIT 1;

In [19]:
for row in c.execute('SELECT 이름, 대출일, 반납일 FROM 도서대출내역 ORDER BY 대출일 DESC LIMIT 1;'):
  print(row)

('문강태', '2020-06-20', '2020-06-25')


#### DISTINCT와 GROUP BY

중복을 제거한다는 의미를 떠올렸을 때, DISTINCT와 GROUP BY의 차이는 무엇일까요? 아래 표를 통해 살펴 보도록 하겠습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_15_2gRlUx0.max-800x600.png)

한 눈에 정리가 되는 것 같네요.

추가로, '기준 별로 중복없이 집계를 하고 싶다!'라고 할 때에는 집계함수와 더불어 DISTINCT, GROUP BY를 모두 활용해야 한답니다. 아래 사진을 참고해주세요.

![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_16_UKpCPxq.png)



# SQL data type

Oracle, SQL Server, PostgreSQL, MySQL, SQLite 등등... SQL의 종류가 다양한 만큼 데이터 타입도 다양하고 조회 및 조작어도 세밀하게 다른데요. 

기본적으로 알아 놓으면 좋은 형식들을 아래에 정리해 보았습니다.

![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_17_sVyiJI8.png)

위 테이블을 보시면 숫자는 거의 INT로 끝납니다. 

INT는 종류별로 그 안에 넣을 수 있는 바이트(bite) 수가 다른데요.

따라서 우리가 넣을 숫자의 범위(최솟값, 최댓값)와 크기가 어느 정도 될지를 생각하여 타입을 정해주는 것을 권장합니다.

문자형과 날짜형도 마찬가지입니다.

데이터의 타입(형)을 바꾸는 실습을 해보도록 하겠습니다.

In [20]:
for row in c.execute('SELECT * FROM 도서대출내역2;'):
  print(row)

('101', '문강태', '2020-06', '20일')
('102', '고문영', '2020-06', '10일')
('103', '문상태', '2020-06', '8일')
('104', '강기둥', '2020-06', '3일')


#### <테이블 명 : 도서대출내역 2>

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_18_owwKcqu.png)

대출자 별 대출 연월 정보와 해당 월에 총 며칠을 대출했는지에 대한 정보가 들어가 있네요.

위 네 명의 사람들의 평균 대출일수 가 궁금하다면 어떻게 해야 할까요? 평균은 AVG() 함수를 사용하면 되는데요.

평균을 구하기 위해서는 일단 대출일수라는 컬럼이 숫자 타입을 지니고 있어야 합니다.

음...🧐 '~일'로 표기되어 있는 것으로 보아 문자형이라고 예상해볼 수 있을 것 같네요.

우선, 각 데이터들의 타입을 확인해 봅시다. sqlite에서는 테이블의 데이터 타입 정의를 확인할 수 있도록 pragma table_info('테이블명') 문을 제공합니다.

이것은 표준 SQL 쿼리가 아니라 데이터베이스마다 달라지는 유틸리티 쿼리라고 할 수 있겠습니다.



In [21]:
import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
c = conn.cursor()

for row in c.execute('pragma table_info(도서대출내역)'):
  print(row)

(0, 'ID', 'varchar', 0, None, 0)
(1, '이름', 'varchar', 0, None, 0)
(2, '도서ID', 'varchar', 0, None, 0)
(3, '대출일', 'varchar', 0, None, 0)
(4, '반납일', 'varchar', 0, None, 0)


데이터 타입을 보니 모두 VARCHAR로 되어 있네요. 그럼 '문자형'인 거겠죠?

먼저, '대출일수' 컬럼을 문자형에서 숫자형으로 변환해 보겠습니다. 그래야 평균을 구할 수 있을 테니까요!

그런데 말입니다.🤔 한 가지 걸리는 부분이 있습니다. '대출일수' 컬럼을 보면 숫자와 함께 '일'이라는 문자가 붙어 있지요? 이 문자를 일단 떼어 내야겠네요.

문자형의 특정 부분을 떼어 내는 함수는 LEFT, RIGHT, SUBSTRING를 사용하면 된답니다.

언어에 따라 SUBSTRING 함수는 substring() 또는 substr()으로 골라서 사용해야 할 수 있으므로 주의해 주세요.

저희는 SQLite를 사용하니까 SUBSTR() 함수를 사용해야 합니다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_19_lGwzIpP.png)

SELECT *, SUBSTR(대출일수, 1, (length(대출일수)-1)) AS 대출일수_수정   
FROM 도서대출내역2;

#### ==================================================
'대출일수'의 마지막 컬럼 값이 항상 '일'로 끝나므로 마지막 자리만 삭제하기 위해 위와 같은 쿼리를 작성하였습니다.

만약 무조건 'OO일'과 같이 3자리라면 앞에서부터 2개만 잘라내면 되겠지만, 20일/10일/8일과 같이 '일'의 앞 자리수가 다른 경우 위처럼 하거나 sqlite에서 제공하는 split_part() 함수를 사용하면 됩니다.



그리고 AS를 사용해 수정한 값을 '대출일수_수정'이라는 컬럼으로 새롭게 저장합니다.

조회해보면 한 개의 컬럼이 맨 뒤에 추가된 것을 확인할 수 있을 거예요.



In [22]:
for row in c.execute('SELECT *, SUBSTR(대출일수, 1, (length(대출일수)-1)) AS 대출일수_수정 FROM 도서대출내역2;'):
  print(row)

('101', '문강태', '2020-06', '20일', '20')
('102', '고문영', '2020-06', '10일', '10')
('103', '문상태', '2020-06', '8일', '8')
('104', '강기둥', '2020-06', '3일', '3')


그 다음은 CAST 함수를 이용하여, 잘라낸 부분에 더하여 숫자로 변환해 보겠습니다.

#### ==================================================

SELECT *, CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT) AS 대출일수_수정   
FROM 도서대출내역2 ;

In [23]:
for row in c.execute('SELECT *, CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT) AS 대출일수_수정 FROM 도서대출내역2 ;'):
  print(row)

('101', '문강태', '2020-06', '20일', 20)
('102', '고문영', '2020-06', '10일', 10)
('103', '문상태', '2020-06', '8일', 8)
('104', '강기둥', '2020-06', '3일', 3)


코드를 보시면 CAST 라는 게 나왔죠? CAST()는 형 변환을 위한 함수입니다.

CAST(형 변환하고 싶은 컬럼명 AS 변환하고 싶은 타입)과 같이 쓰시면 간단히 형 변환을 할 수 있답니다.

이렇게 총 '대출일수_수정' 컬럼을 만들어 냈습니다.

마지막 컬럼은 INT로 바뀌었습니다.

'대출일수_수정' 컬럼의 평균을 구해볼까요? 필요한 컬럼만 가져와 보도록 하겠습니다.

SELECT ID, 이름, 대출년월   
, AVG(CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT)) AS 대출일수_평균   
FROM 도서대출내역2   
GROUP BY 1,2,3;

In [24]:
for row in c.execute('SELECT ID, 이름, 대출년월, AVG(CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT)) AS 대출일수_평균 FROM 도서대출내역2 GROUP BY 1,2,3;'):
  print(row)

('101', '문강태', '2020-06', 20.0)
('102', '고문영', '2020-06', 10.0)
('103', '문상태', '2020-06', 8.0)
('104', '강기둥', '2020-06', 3.0)


집계함수 등을 사용할 때는 GROUP BY를 사용해 주는 것을 유념해 주세요.

또한 GROUP BY 이후에는 컬럼명 또는 숫자를 적어주면 됩니다.

우리가 다룬 함수 외에도 굉장히 다양한 함수들이 존재합니다.

필요할 때마다 구글 검색 등을 통해 찾아 보면서 사용하는 것을 추천드립니다.



#### WHERE 조건에 다양한 조건 입력하기

SELECT * FROM 도서대출내역2   
WHERE ~

위와 같이 기본적인 형태는 아실 텐데요. WHERE 조건절을 몇 개씩 더 할 수도 있습니다.

SELECT * FROM 도서대출내역2   
WHERE   
    조건1   
AND 조건2   
AND 조건3   
AND (조건 4 OR 조건5);


위와 같은 형태로 무수히 많은 조건들을 잇고 이어서, 우리가 원하는 형태로 테이블을 가져올 수 있습니다. 조건절을 잘 사용하는 것이야말로 SQL의 가장 중요한 기본기라고 할 수 있습니다.

예제를 통해, WHERE 조건절 안에서 쓸 수 있는 몇 가지를 살펴보겠습니다.

특정 문자를 포함하는 row를 가져오고 싶을 때

특정 기간 혹은 특정 날짜의 전 또는 이후의 row를 가져오고 싶을 때

특정 숫자 이상 또는 이하의 row를 가져오고 싶을 때

1. 특정 문자를 포함하는 row를 가져오고 싶을 때

    먼저 "특정 문자를 포함하는 row를 가져오고 싶을 때"부터 살펴보겠습니다.

    저는 '도서대출내역 2' 테이블에서 문씨 형제의 결과만 가져오고 싶습니다. 즉, 문강태와 문상태의 결과만 조회하고 싶은 것이죠. 그럴 경우 어떻게 하면 좋을까요?
    
SELECT * FROM 도서대출내역2 
WHERE 이름 LIKE "문%" ;



In [25]:
for row in c.execute('SELECT * FROM 도서대출내역2 WHERE 이름 LIKE "문%" ;'):
  print(row)

('101', '문강태', '2020-06', '20일')
('103', '문상태', '2020-06', '8일')


LIKE는 문자열 컬럼에서 사용할 수 있는 것으로 해당 문자를 포함 또는 해당 문자로 시작 또는 종료하는 것을 불러올 수 있습니다.

위 코드에서 "문%"은 '문'으로 시작하는 모든 문자열을 다 가져오라는 명령이 됩니다.

만약 '%문%'으로 조회를 한다면 이름의 시작, 중간, 끝의 어딘가에 '문'이 존재한다면 모두 가져오라는 명령이 됩니다.

자, 그럼 '%문'으로 조회한다면 어떨까요? '문'으로 끝나는 모든 문자열을 다 가져오라는 명령이 된답니다.

매우 간단하지만 강력한 명령어 중 하나이니 꼭 유념해 주세요!

    주의! 문자열은 꼭 큰따옴표(")를 이용해 감싸주셔야 하는데요. 데이터베이스마다 작은 따옴표, 큰따옴표를 모두 지원하는 경우도 있고 아닌 경우도 있으니 잘 찾아보시기바랍니다.
    
2. 특정 기간 혹은 특정 날짜의 이전 또는 이후의 row를 가져오고 싶을 때

    다음은 '특정 기간 혹은 특정 날짜의 이전 또는 이후의 row를 가져오고 싶을 때'를 살펴보겠습니다. 날짜 관련 데이터가 있을 때 사용하기 좋을 것 같죠?

    자, 도서대출내역 테이블에서 6월 첫째 주에 대출을 한 사람들의 정보만 가져오고 싶다고 해봅시다. 어떻게 하면 될까요?
    
    SELECT * FROM 도서대출내역   
    WHERE 대출일 >= "2020-06-01"   
    AND 대출일 <= "2020-06-07" ;

In [26]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 대출일 >= "2020-06-01" AND 대출일 <= "2020-06-07" ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('102', '고문영', 'bbb', '2020-06-01', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


문강태, 고문영, 문상태 세 사람이 나오네요.

위 코드처럼 간단히 부등호를 사용해 결과를 조회할 수도 있고, BETWEEN이라는 함수를 활용할 수도 있습니다.

    SELECT * FROM 도서대출내역   
    WHERE 대출일 BETWEEN "2020-06-01" AND "2020-06-07" ;
    
    

In [27]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 대출일 BETWEEN "2020-06-01" AND "2020-06-07" ;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('102', '고문영', 'bbb', '2020-06-01', None)
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


ETWEEN의 경우에는 BETWEEN(시작일 AND 종료일)로 범위를 정해 사용할 수 있는데요. 시작일과 종료일을 "포함"한다는 점을 꼭 기억해 주세요!

#### 특정 숫자 이상 또는 이하의 row를 조회하고 싶을 때
이 외에도 부등호를 통해 특정 숫자 이상 또는 이하의 row를 조회하고 싶을 때도 사용할 수 있습니다.

SELECT *   
, CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT) AS 대출일수_수정   
FROM 도서대출내역2   
WHERE  대출일수_수정 > 5 ;

In [28]:
for row in c.execute('SELECT *, CAST(SUBSTR(대출일수, 1, (length(대출일수)-1)) AS INT) AS 대출일수_수정 FROM 도서대출내역2 WHERE 대출일수_수정 > 5 '):
  print(row)

('101', '문강태', '2020-06', '20일', 20)
('102', '고문영', '2020-06', '10일', 10)
('103', '문상태', '2020-06', '8일', 8)


우선 대출일수_수정이라는 컬럼을 만들고, 그 밑 조건절에서 '대출일수'가 5일을 초과하는 사람들의 정보를 가져온다는 것을 확인하실 수 있습니다.

이번에도 문강태, 고문영, 문상태 세 사람이 조회가 되었군요.

#### NULL 조건을 다루는 방법


한 가지 짚고 넘어가 보겠습니다. 바로 NULL(None)인데요.
테이블 내 특정 컬럼 안에 NULL이 들어가 있는 것을 보셨을 거예요.

NULL은 이미 예상하셨겠지만 해당 컬럼에 '정보가 들어 있지 않은 상태'를 나타내는데 사용됩니다.

테이블을 조회하다 보면 NULL만 가져오고 싶거나 NULL을 제외하고 가져오고 싶은 경우가 있을 텐데요. 

그럴 때도 WHERE 조건절을 활용하면 됩니다.

SELECT * FROM 도서대출내역   
WHERE 반납일 IS NOT NULL;

In [29]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 반납일 IS NOT NULL;'):
  print(row)

('101', '문강태', 'aaa', '2020-06-01', '2020-06-05')
('101', '문강태', 'ccc', '2020-06-20', '2020-06-25')
('103', '문상태', 'ccc', '2020-06-01', '2020-06-05')


WHERE 반납일 IS NOT NULL 부분을 살펴 보겠습니다. 

직역하면 '반납일이 NULL이 아닌 것'이라는 뜻이 되겠죠?

반납일이 NULL이라는 것은 미반납 상태 또는 반납일에 대한 정보가 없다는 뜻일 테니, NULL이 아닌 상태는 반납을 한 사람들을 의미하겠네요.

조회해 보니 문강태, 문상태 두 형제의 데이터가 출력되었습니다.

그렇다면 거꾸로 NULL인 사람들 즉, 아직 책을 반납하지 않은 사람(들)이 누구인지 조회해 볼까요?

위 코드에서 NOT만 제외하면 되니 매우 간단하네요.


SELECT * FROM 도서대출내역   
WHERE 반납일 IS NULL ;

In [30]:
for row in c.execute('SELECT * FROM 도서대출내역 WHERE 반납일 IS NULL;'):
  print(row)

('102', '고문영', 'bbb', '2020-06-01', None)
('102', '고문영', 'ddd', '2020-06-08', None)
('104', '강기둥', None, None, None)


고문영, 강기둥 두 사람의 데이터가 출력되었습니다.

자 그럼, 몇 개의 조건을 합쳐서 조회해 볼까요?

- 조건 1 : 이 름이 '태'로 끝나는 데이터를 조회


- 조건 2 : 대출일이 2020년 6월 19일 이후인 데이터를 조회


- 조건 3 : ID가 102 아래인 데이터를 조회


- 조건 4 : 반납일이 NULL이 아닌 데이터를 조회


위의 모든 조건은 AND로 이어서 작성하셨나요? OUTPUT이 한 명의 이름만 조회되었나요?
아마 '문강태'만 조회가 되었다면 여러분은 알맞게 조회한 것이니 안심해도 됩니다. :)



# join

![title](https://d3s0tskafalll9.cloudfront.net/media/images/Untitled_21_ns7QjSV.max-800x600.png)



In [31]:
import os
db_path = os.getenv('HOME')+'/mydb.db'
conn = sqlite3.connect(db_path)  # mydb.db에 연결합니다.
c = conn.cursor()

JOIN을 이해하기 위해서는 아스라이 사라져 있는 '집합'에 대한 개념을 끄집어내야 합니다.

A 테이블과 B 테이블, 총 2개의 테이블이 있다고 가정을 해봅시다. 어떻게 결합해 조회할 수 있을까요? (LEFT TABLE : A 테이블, RIGHT TABLE : B 테이블 )

- INNER JOIN : A 테이블과 B 테이블의 교집합을 조회


- LEFT JOIN : (기준은 A 테이블) A 테이블을 기준으로 해서 B 테이블은 공통되는 부분만 조회


- RIGHT JOIN : (기준은 B 테이블) B 테이블을 기준으로 해서 A 테이블은 공통되는 부분만 조회


- FULL JOIN : A 테이블과 B 테이블 모두에서 빠트리는 부분 없이 모두 조회


- JOIN의 기본 구문과 함께 하나씩 이해해 보도록 하겠습니다.

먼저, INNER JOIN 입니다.

앞에서부터 예제로 계속 사용했던 테이블을 활용해 2개의 테이블을 준비했습니다.

< 대출내역, 도서명 테이블 >

![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_22_kvwSJTr.png)

문강태씨나 고문영씨가 각자 무엇을 빌렸는지가 궁금한데요. '대출내역' 테이블만 보면 어떤 도서를 빌렸는지 알 수가 없겠죠?

이런 경우 2개의 테이블을 서로 JOIN하여 확인해 보도록 하겠습니다.

먼저 대출내역과 도서명 테이블을 각각 조회해서 구조를 파악해보세요.

SELECT * FROM 대출내역 ;   
SELECT * FROM 도서명 ;

In [32]:
print('대출내역 테이블')
for row in c.execute('SELECT * FROM 대출내역;'):
  print(row)
print('')
print('도서명 테이블')
for row in c.execute('SELECT * FROM 도서명;'):
  print(row)

대출내역 테이블
('101', '문강태', 'aaa')
('102', '고문영', 'bbb')
('102', '고문영', 'fff')
('103', '문상태', 'ccc')
('104', '강기둥', None)

도서명 테이블
('aaa', '악몽을 먹고 자란 소년')
('bbb', '좀비아이')
('ccc', '공룡백과사전')
('ddd', '빨간구두')
('eee', '잠자는 숲속의 미녀')


구조를 파악하셨으면 JOIN에서 제일 중요한 'KEY'를 포착하셔야 합니다. 

대체 'KEY'가 무엇일까요? 'KEY'는 JOIN을 수행할 때 두 테이블을 연결할 수 있는 중요한 다리 역할을 하는 컬럼입니다. 

위 두 개의 테이블을 보았을 때, 어떤 'KEY' 기준으로 테이블을 연결할 것인지 혹시 눈치 채셨나요?

'도서ID'가 있네요. 해당 컬럼 기준으로 한번 연결해보도록 하겠습니다.



#### JOIN의 기본 구문

SELECT 컬럼1, 컬럼2, 컬럼3... FROM A테이블 AS A   
{INNER/LEFT/RIGHT/FULL OUTER} JOIN B테이블 AS B    
ON A.결합컬럼 = B.결합컬럼   
WHERE ~

#### 1) INNER JOIN
INNER JOIN은 두 테이블의 교집합을 뱉어주는 명령어였습니다.

즉, 두 테이블에 모두 있는 정보만 들고 오는 것이죠. 한 쪽이라도 해당 정보가 없다면 조회가 불가능합니다.

직접 조회를 해보겠습니다.

SELECT A.*, B.도서명    
FROM 대출내역 AS A    
INNER JOIN 도서명 AS B   
ON A.도서ID = B.도서ID;

In [33]:
query = '''
SELECT A.*, B.도서명 
FROM 대출내역 AS A 
INNER JOIN 도서명 AS B
ON A.도서ID = B.도서ID;
'''
for row in c.execute(query):
  print(row)

('101', '문강태', 'aaa', '악몽을 먹고 자란 소년')
('102', '고문영', 'bbb', '좀비아이')
('103', '문상태', 'ccc', '공룡백과사전')


![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_23_t85sid9.png)

머릿속으로 그려보면, 도서ID를 기준으로 합친다고 했을 때 공통되는 부분은 'aaa, bbb, ccc' 세 개입니다.

따라서 아래와 같이 결과가 나올 것입니다.



![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_24_cM7OvJd.png)

#### 2) LEFT JOIN

SELECT A.*, B.도서명   
FROM 대출내역 AS A    
LEFT JOIN 도서명 AS B   
ON A.도서ID = B.도서ID;

위와 똑같이 하되, LEFT JOIN으로만 바꿔서 조회하면 어떻게 될까요?

LEFT JOIN은 왼쪽 A 테이블을 기준으로 오른쪽 테이블을 붙이는 것이라고 했었죠?

즉, A 테이블에 있는 데이터는 모두 가져오고 B 테이블과 공통되는 부분만 오른쪽에 붙이게 됩니다.



In [34]:
query = '''
SELECT A.*, B.도서명 
FROM 대출내역 AS A 
LEFT JOIN 도서명 AS B
ON A.도서ID = B.도서ID;
'''
for row in c.execute(query):
  print(row)

('101', '문강태', 'aaa', '악몽을 먹고 자란 소년')
('102', '고문영', 'bbb', '좀비아이')
('102', '고문영', 'fff', None)
('103', '문상태', 'ccc', '공룡백과사전')
('104', '강기둥', None, None)


![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_25_VizrJfJ.png)

짠! 위와 같이 예상하셨나요? 기준이 된 대출내역 테이블은 바뀐 부분 없이, 도서명만 오른쪽에 붙은 것을 확인할 수 있습니다. 

도서명에 도서ID 'fff'가 없으므로 세 번째 줄은 NULL이 생성되었고, 강기둥의 도서ID는 NULL이므로 도서명도 NULL이 됩니다.

    (주의) SQLite에서 실습할 때, 아래 소개할 RIGHT JOIN과 FULL OUTER JOIN은 아래와 같은 에러가 발생할 것입니다.

    OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

현재 SQLite에서는 RIGHT JOIN과 FULL OUTER JOIN를 지원하지 않는가 봅니다. 

하지만 MYSQL 등 주요 RDBMS에서는 이런 형태의 JOIN도 지원되므로, 다음의 두 개념도 같이 숙지해 두시면 좋을 것 같습니다.



#### 3) RIGHT JOIN

SELECT B.*, A.ID, A.이름   
FROM 대출내역 AS A   
RIGHT JOIN 도서명 AS B  
ON A.도서ID = B.도서ID;

이번에는 RIGHT JOIN을 살펴 보겠습니다. 아까 LEFT JOIN에서는 A 테이블이 기준이 되었으니 이번에는 B 테이블이 기준이 될 것 같지 않나요?🤭

In [35]:
query = '''
SELECT B.*, A.ID, A.이름 
FROM 대출내역 AS A 
RIGHT JOIN 도서명 AS B
ON A.도서ID = B.도서ID;
'''
for row in c.execute(query):
  print(row)

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_26_SpMa9st.png)

에러가 발생하였나요? 만약 다른 RDBMS 프로그램을 사용한다면 오른쪽 테이블을 기준으로 해서 왼쪽 테이블 중 공통인 것만 붙고 나머지는 전부 NULL처리가 되었을 겁니다.

#### 4) FULL OUTER JOIN

SELECT A.*, B.도서명  
FROM 대출내역 AS A   
FULL OUTER JOIN 도서명 AS B  
ON A.도서ID = B.도서ID  
ORDER BY 도서ID;



In [36]:
query = '''
SELECT A.*, B.도서명 
FROM 대출내역 AS A 
FULL OUTER JOIN 도서명 AS B
ON A.도서ID = B.도서ID
ORDER BY 도서ID;
'''
for row in c.execute(query):
  print(row)

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

![title](https://d3s0tskafalll9.cloudfront.net/media/original_images/Untitled_27_2Y2zQ6V.png)

이번에도 에러가 발생하였나요? 만약 다른 RDBMS 프로그램을 사용한다면 도서ID를 기준으로 붙을 수 있는 것은 모두 붙고 나머지는 NULL 값이 좌우로 붙은 것을 확인할 수 있습니다.

#### 5) 중첩 질의(Nested Query)

SELECT C.이름, COUNT(*) 대출건수   
FROM (   
	SELECT A.*, B.도서명    
	FROM 대출내역 AS A    
	LEFT JOIN 도서명 AS B   
	ON A.도서ID = B.도서ID ) C   
GROUP BY C.이름;



In [37]:
query = '''
SELECT C.이름, COUNT(*) 대출건수
FROM (
	SELECT A.*, B.도서명 
	FROM 대출내역 AS A 
	LEFT JOIN 도서명 AS B
	ON A.도서ID = B.도서ID ) C
GROUP BY C.이름;
'''
for row in c.execute(query):
  print(row)

('강기둥', 1)
('고문영', 2)
('문강태', 1)
('문상태', 1)


이번에는 좀 복잡한 것 같지만 잘 보면 간단합니다. 

위에서 다룬 LEFT JOIN을 사용한 서브쿼리의 결과 C로부터 다시 쿼리를 수행하는 형태로 쿼리가 중첩되어 있습니다.

이렇게 쿼리를 중첩하는 구조를 이해하고 나면 JOIN과 더불어 데이터베이스를 활용하는 가장 강력한 무기를 얻게 됩니다.